# Tutorial of in-sample resolution enhancement

This tutorial demonstrates in-sample resolution enhancement using a Xenium lung adenocarcinoma data E1. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import scanpy as sc
from MicroMap import MicroMap

## Download data from Google Drive

We first download the required data from Google Drive, which will automatically create a local `data/` directory and store all related files.

In [2]:
import os

# Folder link
GDRIVE_URL = "https://drive.google.com/drive/folders/1jCCmvksz9r1DC-W-2mpXzNqpU9c4Ab8R"
DATA_DIR = "data"

# Create data directory if not exists
os.makedirs(DATA_DIR, exist_ok=True)

# Install gdown if needed
try:
    import gdown
except ImportError:
    !pip install -q gdown
    import gdown

# Download entire folder
gdown.download_folder(GDRIVE_URL, output=DATA_DIR, quiet=False)

print("\n Data download finished!")
print("Files in data/:", os.listdir(DATA_DIR))


Retrieving folder contents


Processing file 195ERY62DFTQHhjlTaex9hWhlcxmPkysE adata_gt_E1.h5ad
Processing file 1wX2jJPV4PjSveY3zjP6328TNzpY9KmIf adata_visium_E1.h5ad
Processing file 1Gaem6GsJfNGo7nffEwRUa2k0Xg6HtlL_ adata_visium_rep2.h5ad
Processing file 1LFOSWH5xWUuBTd89Uwt-lIS3ao4dv8Uz HE_E1.png
Processing file 1y0rxIoiHMaAO7WwDgixjY9UjZHBM_4xD HE_rep2.png


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=195ERY62DFTQHhjlTaex9hWhlcxmPkysE
From (redirected): https://drive.google.com/uc?id=195ERY62DFTQHhjlTaex9hWhlcxmPkysE&confirm=t&uuid=4650cce3-4887-412f-b18f-0d07a7ed9821
To: /data/yyyu/test/imint/tutorial/data/adata_gt_E1.h5ad
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 266M/266M [00:27<00:00, 9.52MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wX2jJPV4PjSveY3zjP6328TNzpY9KmIf
To: /data/yyyu/test/imint/tutorial/data/adata_visium_E1.h5ad
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.34M/4.34M [00:02<00:00, 1.65MB/s]
Downloading.


 Data download finished!
Files in data/: ['adata_gt_rep1.h5ad', 'HE_rep2.png', 'adata_visium_E2.h5ad', 'adata_gt_E1.h5ad', 'adata_visium_rep2.h5ad', 'adata_visium_E1.h5ad', 'HE_E1.png']


Download completed


## Load low resolution data

Next, we load the low-resolution synthetic gene expression data, which have high-resolution ground-truth.

In [3]:
adata = sc.read('./data/adata_visium_E1.h5ad')
adata

AnnData object with n_obs × n_vars = 3496 × 289
    obs: 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'n_counts'
    var: 'n_cells'
    obsm: 'spatial'

## Construct and run with MicroMap object

We then initialize a MicroMap object and compute spot-specific latent priors.

In [4]:
# construct 
micro =  MicroMap( device = 'cuda:0', out_path='in_sample')

# calculate expression latent distrobution prior for each spot
micro.cal_spot_prior(adata.X)

Preprocessing spot level gene expression with 3496 spots and 289 genes ... 
Calculating spot specific prior ... 


  0%|▉                                                                                                                                                                                        | 1/200 [00:01<05:54,  1.78s/it]

[Pretrain Epoch 0] total loss = 117.9052


  1%|█▊                                                                                                                                                                                       | 2/200 [00:02<03:59,  1.21s/it]

[Pretrain Epoch 1] total loss = 95.0064


...

[Pretrain Epoch 197] total loss = 49.7085


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 199/200 [02:47<00:00,  1.19it/s]

[Pretrain Epoch 198] total loss = 49.7815


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [02:48<00:00,  1.19it/s]

[Pretrain Epoch 199] total loss = 49.7360


Image features are extracted using the pathology foundation model UNI, which may take approximately 20 minutes depending on hardware and system configuration. Users are required to download the pretrained model weights pytorch_model.bin in advance from:
https://github.com/mahmoodlab/UNI

In [5]:
# calculate image features via foundation model UNI (https://github.com/mahmoodlab/UNI)
micro.cal_token_feats(   img_path = './data/HE_E1.png',
                         UNI_path = '/data/yyyu/test/UNI/code_raw/assets/ckpts/vit_large_patch16_224.dinov2.uni_mass100k/pytorch_model.bin',
                         scale = 0.4
                         )

Creating and loading model ... 
Extracting features... 
Saving features ...
Saved to: in_sample/features.pt


Subsequently, the main model is trained to predict high-resolution gene expression.

In [6]:
# train the backbone model
micro.train( 
             spot_coord = adata.obsm['spatial'][:,[1,0]], 
             spot_radius = 137.5,
             spot_count = adata.X,
             genes = adata.var_names.tolist()
          )

  0%|▌                                                                                                                                                                                        | 1/300 [00:02<14:44,  2.96s/it]

Epoch 1: total=15.5457, nb=7.6790, size=1.7011, smooth=1.4499, cos_gene=2.6878, cos_cell=2.4824, kl_token=4.7758


  1%|█▏                                                                                                                                                                                       | 2/300 [00:06<15:22,  3.09s/it]

Epoch 2: total=7.8074, nb=3.3146, size=0.6418, smooth=0.8576, cos_gene=1.5451, cos_cell=1.4270, kl_token=3.9130


  1%|█▊                                                                                                                                                                                       | 3/300 [00:09<15:10,  3.07s/it]

...

Epoch 298: total=2.9967, nb=1.6552, size=0.0036, smooth=0.1507, cos_gene=0.2490, cos_cell=0.2055, kl_token=1.9825


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 299/300 [15:48<00:03,  3.21s/it]

Epoch 299: total=2.9952, nb=1.6555, size=0.0032, smooth=0.1510, cos_gene=0.2486, cos_cell=0.2045, kl_token=1.9759


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [15:51<00:00,  3.17s/it]

Epoch 300: total=2.9945, nb=1.6505, size=0.0036, smooth=0.1508, cos_gene=0.2486, cos_cell=0.2063, kl_token=1.9762


In [7]:
# prediction
adata_hir = micro.predict(  genes = adata.var_names.tolist(), extract_mask=False)
adata_hir # High-resolution gene expression across the entire H&E image

AnnData object with n_obs × n_vars = 687960 × 289
    obs: 'patch_y', 'patch_x'
    obsm: 'spatial', 'latent'

## Evaluation

Finally, we load the 8 μm ground-truth data and evaluate the prediction performance using multiple quantitative metrics, including Pearson correlation coefficient (PCC), Spearman rank correlation coefficient (RCC), root mean squared error (RMSE), and structural similarity index measure (SSIM).

In [8]:
from MicroMap.evaluate import evaluation_func, normalize_count

In [9]:
adata_gt = sc.read('./data/adata_gt_E1.h5ad')
adata_gt

AnnData object with n_obs × n_vars = 212081 × 289
    obs: 'patch_y', 'patch_x', 'n_counts'
    var: 'moranI'
    obsm: 'patch', 'spatial'

In [10]:
genes = adata_gt.var_names
tokens = adata_gt.obs_names
adata_use = adata_hir[tokens, genes].copy()
adata_use

expr_gt = adata_gt.to_df()
expr_gt_norm = normalize_count(expr_gt)

expr_use = adata_use.to_df()
expr_use_norm = normalize_count(expr_use)

In [ ]:
patch_shape = (adata_gt.obs[['patch_x', 'patch_y']].max()[0]+1, adata_gt.obs[['patch_x', 'patch_y']].max()[1]+1)
index_gt = adata_gt.obs[['patch_x', 'patch_y']].values
index_gt = (index_gt[:,0], index_gt[:,1])

metric = evaluation_func(expr_use_norm, expr_gt_norm, genes,
                         do_spearman=True, spearman_subsample=100000, spearman_jobs=8,
                         do_ssim=True, ssim_kwargs={'index_gt':index_gt, 'patch_shape':patch_shape})

moranI = adata_gt.var['moranI']
metric[moranI>0.2].median()

PCC     0.520389
RMSE    0.044397
RCC     0.360074
SSIM    0.880514
dtype: float64